In [20]:
import cv2
import mediapipe as mp
import pickle
import math
import os
import numpy as np
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import pandas as pd
import matplotlib.pyplot as plt

In [21]:
# media pipe pose 전역변수로 설정
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True,min_detection_confidence=0.3, model_complexity=0)
mp_drawing = mp.solutions.drawing_utils

In [22]:
# 포즈 감지 함수
def detectPose(image,pose,display=True):
    # input 이미지 복사
    output_image = image.copy()
    # RGB로 변환
    image_BGR = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    # 렌드마크 찍기
    image_BGR.flags.writeable = False
    results = pose.process(image_BGR)
    image_BGR.flags.writeable = True
    
    # 사진 사이즈 구하기
    height, width, _ = image.shape
    # 렌드마크 값 리스트 만들기
    landmarks = []
    visible = []
    if results.pose_landmarks:
        # 렌드마크 그리기
        mp_drawing.draw_landmarks(image=output_image, landmark_list=results.pose_landmarks, connections=mp_pose.POSE_CONNECTIONS)
        #렌드마크 정규값 landmarks 리스트에 넣기
        for landmark in results.pose_landmarks.landmark:
            landmarks.append((int(landmark.x*width), int(landmark.y*height), (landmark.z*width)))
            # visible 값 따로 리스트에 넣기 
            visible.append(landmark.visibility)
    # 사진 결과 보기
    if display:
        plt.figure(figsize=[22,22])
        plt.subplot(121);plt.imshow(image[:,:,::-1]);plt.title("Original Image");plt.axis('off');
        plt.subplot(122);plt.imshow(output_image[:,:,::-1]);plt.title("Output Image");plt.axis('off');
        # 3차원으로도 봐보기
        mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)
    else:
        
        return output_image, landmarks , visible

In [23]:
# 각도 구하는 함수
def calculateAngle(landmark1, landmark2, landmark3):
    x1,y1,_ = landmark1 # z 좌표는 사용 x
    x2,y2,_ = landmark2
    x3,y3,_ = landmark3
    # 3점의 각도 -> 선 2개 -> 각도
    angle = math.degrees(math.atan2(y3-y2, x3-x2) - math.atan2(y1-y2,x1-x2))

    angle = np.abs(angle)
    # 음수의 각도가 나오면 양수로 바꾸기
    if angle >180.0:
        angle = 360 - angle
        
    return angle

In [24]:
# 중앙 각도 구하는 함수
def center_Landmark(landmarks1,landmarks2):
    x1,y1,_ = landmarks1
    x2,y2,_ = landmarks2
    x_cen = (x1+x2)/2
    y_cen = (y1+y2)/2
    return x_cen,y_cen,_

In [25]:
# 테스트 할 영상 경로 넣으면 됨
video = cv2.VideoCapture('./test_video3.mp4')
df = pd.DataFrame() # predict에 넣어 줄 X_test
actions = ['bend', 'headup', 'right', 'waistup']

while video.isOpened():
    # Read a frame.
    ok, frame = video.read()
    # Check if frame is not read properly.
    if not ok:
        # Break the loop.
        break
    # display -> False 넣어서 img랑 렌드마크 받기
    output_img, landmarks,visible = detectPose(frame,pose,display=False)

    if landmarks:
        # visible 가져오기
        left_shoulder = visible[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
        right_shoulder = visible[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
        left_hip = visible[mp_pose.PoseLandmark.LEFT_HIP.value]
        right_hip = visible[mp_pose.PoseLandmark.RIGHT_HIP.value]
        left_knee = visible[mp_pose.PoseLandmark.LEFT_KNEE.value]
        right_knee = visible[mp_pose.PoseLandmark.RIGHT_KNEE.value]
        left_ankle = visible[mp_pose.PoseLandmark.LEFT_ANKLE.value]
        right_ankle = visible[mp_pose.PoseLandmark.RIGHT_ANKLE.value]
        right_pinky = visible[mp_pose.PoseLandmark.RIGHT_PINKY.value]
        left_pinkey = visible[mp_pose.PoseLandmark.LEFT_PINKY.value]
        # 어깨 - 엉덩이 - 무릎 => 올리는 동작 각도 구하기 위함 -> Main angle
        if left_hip>right_hip:
            main_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                                landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                                landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value])
        else:
            main_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                                landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                                landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value])                                                    
        
        # 엉덩이 - 무릎 - 발목 => 무릎 꺾였는지 보기 위함
        if left_knee > right_knee:
            knee_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                            landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value],
                                            landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value])
        else:
            knee_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                            landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value],
                                            landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value])
                                            
        # 어깨 허리  중앙값
        cen_shoulder = center_Landmark(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                    landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value])
        cen_hip = center_Landmark(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value])
    
        # 코 - 어깨 - 엉덩이 => 목 각도
        head_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.NOSE.value],
                                    cen_shoulder,cen_hip)

        # 잘보이는 새끼손가락의 y 좌표
        if right_pinky > left_pinkey:
            y_pinky = landmarks[mp_pose.PoseLandmark.RIGHT_PINKY.value][1]
        else:
            y_pinky = landmarks[mp_pose.PoseLandmark.LEFT_PINKY.value][1]

        # 엉덩이 중앙의 y좌표와 새끼손가락의 y 좌표사이의 거리 
        waist_len = cen_hip[1]-cen_shoulder[1]

        globals()['list'] = knee_angle,head_angle,waist_len
        df = df.append([globals()[f'list']])
        print(globals()['list'])
        loaded_model = pickle.load(open('03.Knn_model', 'rb'))
        result = loaded_model.predict_proba([globals()['list']])
        print(result)
        # 각 회마다 맞는 자세인지 매핑하기
        if result.argmax() ==0:
            cv2.putText(output_img, actions[0], (120,200),cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
        elif result.argmax() ==1:
            cv2.putText(output_img, actions[1], (120,200),cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
        elif result.argmax() == 2:
            cv2.putText(output_img, actions[2], (120,200),cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
        elif result.argmax() ==3:
            cv2.putText(output_img, actions[3], (120,200),cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
        
    cv2.imshow('img',output_img)
    cv2.waitKey(10)
video.release()
cv2.destroyAllWindows()


C:\Users\smhrd\AppData\Local\Temp\ipykernel_15348\2774937686.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([globals()[f'list']])
C:\Users\smhrd\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\smhrd\AppData\Local\Temp\ipykernel_15348\2774937686.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([globals()[f'list']])
C:\Users\smhrd\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\smhrd\AppData\Local\Temp\ipykernel_15348\2774937686.py:68: FutureWarning: The frame.append method is de

(59.34304524517642, 150.56214994469173, -41.0)
[[1. 0. 0. 0.]]
(55.59819447873041, 150.9674900585958, -33.5)
[[1. 0. 0. 0.]]
(52.01728190538944, 150.51302970533655, -36.5)
[[1. 0. 0. 0.]]
(51.80131482398157, 153.18740832052922, -34.0)
[[1. 0. 0. 0.]]


C:\Users\smhrd\AppData\Local\Temp\ipykernel_15348\2774937686.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([globals()[f'list']])
C:\Users\smhrd\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\smhrd\AppData\Local\Temp\ipykernel_15348\2774937686.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([globals()[f'list']])
C:\Users\smhrd\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\smhrd\AppData\Local\Temp\ipykernel_15348\2774937686.py:68: FutureWarning: The frame.append method is de

(51.91006080459521, 153.2229950623735, -32.5)
[[1. 0. 0. 0.]]
(52.401159070957334, 152.56540966891785, -31.5)
[[1. 0. 0. 0.]]
(52.118725639713944, 151.9975920715037, -35.0)
[[1. 0. 0. 0.]]
(51.74163874613831, 149.15908889403917, -36.5)
[[1. 0. 0. 0.]]


C:\Users\smhrd\AppData\Local\Temp\ipykernel_15348\2774937686.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([globals()[f'list']])
C:\Users\smhrd\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\smhrd\AppData\Local\Temp\ipykernel_15348\2774937686.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([globals()[f'list']])
C:\Users\smhrd\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\smhrd\AppData\Local\Temp\ipykernel_15348\2774937686.py:68: FutureWarning: The frame.append method is de

...

(173.23799568565056, 131.8943195889173, 208.5)
[[0.33333333 0.         0.66666667 0.        ]]
(175.767773514767, 135.58645167258098, 210.0)
[[0.33333333 0.         0.66666667 0.        ]]
(176.52353848726415, 132.13495857620612, 203.5)
[[0.33333333 0.         0.66666667 0.        ]]
(176.85484228687946, 137.91051123113587, 203.0)
[[0.33333333 0.         0.66666667 0.        ]]


C:\Users\smhrd\AppData\Local\Temp\ipykernel_15348\2774937686.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([globals()[f'list']])
C:\Users\smhrd\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\smhrd\AppData\Local\Temp\ipykernel_15348\2774937686.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([globals()[f'list']])
C:\Users\smhrd\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\smhrd\AppData\Local\Temp\ipykernel_15348\2774937686.py:68: FutureWarning: The frame.append method is de

(3.2976324351724213, 140.93534777050476, 196.0)
[[0.66666667 0.         0.33333333 0.        ]]
(60.48744092057772, 145.58998372028196, 187.5)
[[0.66666667 0.         0.33333333 0.        ]]
(91.05496717567007, 149.76259107600217, 183.5)
[[0.66666667 0.         0.33333333 0.        ]]
(89.45692830627623, 149.09896637235423, 184.0)
[[0.66666667 0.         0.33333333 0.        ]]


C:\Users\smhrd\AppData\Local\Temp\ipykernel_15348\2774937686.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([globals()[f'list']])
C:\Users\smhrd\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\smhrd\AppData\Local\Temp\ipykernel_15348\2774937686.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([globals()[f'list']])
C:\Users\smhrd\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\smhrd\AppData\Local\Temp\ipykernel_15348\2774937686.py:68: FutureWarning: The frame.append method is de

(78.6619710322098, 149.4075834148156, 188.0)
[[0.66666667 0.         0.33333333 0.        ]]
(164.63196372324825, 142.57120677452713, 195.0)
[[0.33333333 0.         0.66666667 0.        ]]
(150.2967327811531, 142.63041002483735, 206.0)
[[0.66666667 0.         0.33333333 0.        ]]


In [26]:
df

,0,1,2
0,59.343045,150.562150,-41.0
0,55.598194,150.967490,-33.5
0,52.017282,150.513030,-36.5
0,51.801315,153.187408,-34.0
0,51.910061,153.222995,-32.5
...,...,...,...
0,91.054967,149.762591,183.5
0,89.456928,149.098966,184.0
0,78.661971,149.407583,188.0
0,164.631964,142.571207,195.0


In [27]:
# 모델 불러서 result에 예측 하기
loaded_model = pickle.load(open('03.Knn_model', 'rb'))
result = loaded_model.predict(df)
proba = loaded_model.predict_proba(df)


C:\Users\smhrd\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\smhrd\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


In [28]:
len(result)

697

In [30]:
proba

array([[1.        , 0.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        , 0.        ],
       ...,
       [0.66666667, 0.        , 0.33333333, 0.        ],
       [0.33333333, 0.        , 0.66666667, 0.        ],
       [0.66666667, 0.        , 0.33333333, 0.        ]])

In [31]:
# 각 회마다 맞는 자세인지 매핑하기
val=[]
actions = ['bend',  'headup', 'right', 'waistup']
for i in range(len(result)):
    if result[i] ==0:
        val.append(actions[0])
    elif result[i] ==1:
        val.append(actions[1])
    elif result[i] == 2:
        val.append(actions[2])
    elif result[i] ==3:
        val.append(actions[3])
    
    print(f'{val[i]}, 정확도 : {proba[i]}')

bend, 정확도 : [1. 0. 0. 0.]
bend, 정확도 : [1. 0. 0. 0.]
bend, 정확도 : [1. 0. 0. 0.]
bend, 정확도 : [1. 0. 0. 0.]
bend, 정확도 : [1. 0. 0. 0.]
bend, 정확도 : [1. 0. 0. 0.]
bend, 정확도 : [1. 0. 0. 0.]
bend, 정확도 : [1. 0. 0. 0.]
bend, 정확도 : [1. 0. 0. 0.]
bend, 정확도 : [1. 0. 0. 0.]
bend, 정확도 : [1. 0. 0. 0.]
bend, 정확도 : [1. 0. 0. 0.]
bend, 정확도 : [1. 0. 0. 0.]
bend, 정확도 : [1. 0. 0. 0.]
bend, 정확도 : [1. 0. 0. 0.]
bend, 정확도 : [1. 0. 0. 0.]
bend, 정확도 : [1. 0. 0. 0.]
bend, 정확도 : [1. 0. 0. 0.]
bend, 정확도 : [1. 0. 0. 0.]
bend, 정확도 : [1. 0. 0. 0.]
bend, 정확도 : [1. 0. 0. 0.]
bend, 정확도 : [1. 0. 0. 0.]
bend, 정확도 : [1. 0. 0. 0.]
bend, 정확도 : [1. 0. 0. 0.]
bend, 정확도 : [1. 0. 0. 0.]
bend, 정확도 : [1. 0. 0. 0.]
bend, 정확도 : [1. 0. 0. 0.]
bend, 정확도 : [1. 0. 0. 0.]
bend, 정확도 : [1. 0. 0. 0.]
bend, 정확도 : [1. 0. 0. 0.]
bend, 정확도 : [1. 0. 0. 0.]
bend, 정확도 : [1. 0. 0. 0.]
bend, 정확도 : [1. 0. 0. 0.]
bend, 정확도 : [1. 0. 0. 0.]
bend, 정확도 : [1. 0. 0. 0.]
bend, 정확도 : [1. 0. 0. 0.]
bend, 정확도 : [1. 0. 0. 0.]
bend, 정확도 : [1. 0. 0. 0.]
bend, 정확도 : 